In [1]:
%display latex
from pseries_basis import *
from ore_algebra import *
n = PSBasis.n(PSBasis)

In [2]:
B = BinomialBasis(); B

Binomial basis (x) choose n

In [3]:
OE.<E> = OreAlgebra(QQ[x], ('E', lambda p : p(x=x+1), lambda p : 0))
x = OE.base().gens()[0]

In [95]:
def poly_decomp(polynomial):
    r'''
        Method that splits a polynomial into a lists of monomials and a list of coefficients indexed in 
        such a way that summing thorugh both lists gives the original polynomial.
        
        This method unifies the behavior or Univariate and Multivariate polynomials.
    '''
    from sage.rings.polynomial.polynomial_element import is_MPolynomialRing, is_PolynomialRing
    if is_PolynomialRing(polynomial.parent()):
        g = polynomial.parent().gens()[0]
        d = polynomial.degree()
        monomials = [g**i for i in range(d+1)]
        coefficients = polynomial.coefficients(False)
        # we clean the zeros to have a sparse representation
        monomials = [monomials[i] for i in range(d+1) if coefficients[i] != 0]
        coefficients = [c for c in coefficients if c != 0]
    elif is_MPolynomialRing(polynomial.parent()):
        monomials = polynomial.monomials()
        coefficients = polynomial.coefficients()
    else:
        raise TypeError("The input must be a polynomial")
    return monomials, coefficients

def apply_operator_to_seq(operator, seq):
    r'''
        Method to apply an operator to a sequence.
        
        This method will be similar to the usual __call__ method from ``ore_algebra``, but with sequences that
        are given as functions.
        
        INPUT:
        
        * ``operator``: and operator with 1 generator (the shift) over the polynomial ring `\mathbb{R}[x]`.
        * ``seq``: a sequence in functional format. This means that we can call it with integer values and we 
          obtain the values of the sequence at each point.
          
        OUTPUT:
        
        A sequence in function format.
    '''
    if len(operator.parent().gens()) > 1:
        raise TypeError()
    E = operator.parent().gens()[0]
    v = operator.parent().base().gens()[0]
    coefficients = operator.coefficients(sparse=False)
    return lambda i : sum(coefficients[j](**{str(v):i})*seq(i+j) for j in range(len(coefficients)))

def get_converted_init(seq, bas, size):
    r'''
        Method that computes new initial values after a conversion.
        
        Let `((P_{n,k})_n)_k` be a basis of the sequence space and ``seq`` the
        functional representation of a sequence `(a_n)_n`. If we express this
        sequence in the form:
        
        .. MATH::
            
            a_n = \sum_{k=0}^n P_{n,k} c_k,
            
        then the sequence `(c_k)_k` can be explicitly computed by solving a linear system.
        
        This method solves such system for a fixed amount of elements (gibven by ``size``)
    '''
    inhom = vector([seq(i) for i in range(size)])
    M = Matrix([[bas[i](j) for i in range(size)] for j in range(size)])
    return list(M.solve_right(inhom))
    
def solution(operator, init):
    d = operator.order()
    required = max(
                    -min([el[0] for el in cat_bin.polynomial().lc().roots() if el[0] in ZZ]),
                    -min([el[0] for el in cat_bin.polynomial().lc().roots() if el[0] in ZZ]), 
                    d)
    if len(init) < required:
        raise ValueError(f"More data ({required}) is needed")
    @lru_cache
    def __aux_sol(n):
        if n < 0:
            return 0
        elif n < operator.order():
            return init[n]
        else:
            coeffs = operator.polynomial().coefficients()
            lc = coeffs.pop()
            return -sum(__aux_sol(n-d+i)*coeffs[i](n-d) for i in range(operator.order()))/lc(n-d)
    return __aux_sol

def eval_ore_operator(operator, ring=None,**values):
    r'''
        Method to evaluate ore operators
        
        This method evaluate operators from ``ore_algebra`` as they are polynomials. This allows to change the name 
        of the generators to try a iterative approach.
    '''
    gens = [str(el) for el in operator.parent().gens()]
    outer_vals = {el : values.get(el, 0) for el in gens}
    inner_vals = {el : values[el] for el in values if (not el in outer_vals)}
    monomials,coefficients = poly_decomp(operator.polynomial())
    coefficients = [el(**inner_vals) for el in coefficients]
    monomials = [prod(
        outer_vals[str(g)]**(m.degree(g)) for g in operator.polynomial().parent().gens()
    ) for m in monomials]
    result = sum(coefficients[i]*monomials[i] for i in range(len(monomials)))
    if ring != None:
        return ring(result)
    return result

class latex_str:
    def __init__(self, data):
        self.__data = data
        self.__latex = None
        
    def __repr__(self):
        return self.__data
    def __str__(self):
        return self.__data
    def _latex_(self):
        if self.__latex is None:
            self.__latex = self.__data.replace("*", "").replace("+0", "")
            
        return self.__latex

def print_recurrence(operator):
    r'''
        Method to print nicely the recurrence equation induced by an operator.
    '''
    if len(operator.parent().gens()) > 1:
        raise TypeError()
    E = operator.parent().gens()[0]
    v = operator.parent().base().gens()[0]
    coefficients = operator.coefficients(sparse=False)
    
    return latex_str(" + ".join(
        "(" + 
        str(coefficients[j]).replace(str(v),'n') + 
        ")*"
        "a_{n+" + str(j) + "}" for j in range(len(coefficients))) + " = 0")

In [96]:
# Catalan sequence
cat = lambda n : catalan_number(n)
cat_op = (x+2)*E - (4*x + 2)

In [99]:
cat_bin = eval_ore_operator(B.remove_Sni(B.recurrence(cat_op)), OE, Sn=E, n=x,Sni=1); show(cat_bin)
cat_bin_bin = eval_ore_operator(B.remove_Sni(B.recurrence(cat_bin)), OE, Sn=E, n=x,Sni=1); cat_bin_bin

(x + 3)*E^2 + (-2*x - 2)*E - 3*x - 3

(x + 4)*E^3 + (x + 5)*E^2 + (-4*x - 6)*E - 4*x - 4

In [107]:
in_cat = solution(cat_bin, get_converted_init(cat, B, 3))
in_in_cat = solution(cat_bin_bin, get_converted_init(in_cat, B, 4))

In [109]:
[in_in_cat(i) for i in range(10)]

[1, -1, 2, -3, 6, -10, 20, -35, 70, -126]

In [110]:
[cat(i) == sum(sum(in_in_cat(k2)*binomial(k,k2) for k2 in range(k+1))*binomial(i,k) for k in range(i+1)) for i in range(10)]

[True, True, True, True, True, True, True, True, True, True]

$$a_0 = \binom{0}{0} c_0 = c_0$$
$$a_1 = \sum_{k=0}^1 \binom{1}{k} c_k = \binom{1}{0} c_0 + \binom{1}{1} c_1 = c_0 + c_1$$

In [112]:
[cat(i) for i in range(10)]

[1, 1, 2, 5, 14, 42, 132, 429, 1430, 4862]

In [169]:
B.remove_Sni((B.recurrence(cat_op)))

(n + 3)*Sn^2 + (-2*n - 2)*Sn - 3*n - 3

In [181]:
cat_bin

(-3*x - 3)*E^2 + (-2*x - 2)*E + x + 3

In [182]:
v = vector([1,2,3])
M = Matrix([[1,2,3],[4,5,6],[7,8,9]])

In [185]:
M*M.solve_right(v) == v

True

$$f(x) \in \mathbb{K}[[x]],\qquad f(x) = \sum_n c_n P_n(x)$$

$$(f_n)_n \in \mathbb{K}^\mathbb{N}$$

In [8]:
B

Binomial basis (x) choose n

In [22]:
[x*B[i] - (i+1)*B[i+1] - i*B[i] for i in range(2,10)]

[0, 0, 0, 0, 0, 0, 0, 0]

In [20]:
B[5]*5

1/24*x^5 - 5/12*x^4 + 35/24*x^3 - 25/12*x^2 + x

In [19]:
24/120

1/5

In [87]:
AUX = PolynomialRing(QQ, 'p')

In [92]:
poly = AUX([1,2,3])
monomials = poly.monomials(); monomials.reverse()
coefficients = poly.coefficients(sparse=False)

In [94]:
sum(monomials[i]*coefficients[i] for i in range(len(monomials))) == poly

True

In [79]:
AUX = PolynomialRing(QQ, 'a,b')

In [83]:
poly = AUX.random_element(3)

In [189]:
def search_dfinite_order(min_order, min_results = 5):
    import re
    
    i = 1
    results = []
    
    while(len(results) < min_results):
        print(f"Cathcing {i*100} first sequences")
        sequences = oeis("D-finite", max_results = 100, first_result = (i-1)*100)
        print(f"Analyzing...")
        print("Found orders ", end = "")
        for sequence in sequences:
            formulas = [el for el in sequence.formulas() if el.find("D-finite") >= 0]
            for formula in formulas:
                start_pos = formula.find("D-finite with recurrence: ") + len("D-finite with recurrence: ")
                end_pos = max(formula.find(".", start_pos), len(formula))
                formula = formula[start_pos:end_pos+1]
                
                n = var('n')
                arguments = [eval(el)-n for el in re.findall(r"a\(([^\)]*)\)", formula)]
                arguments = [ZZ(el) for el in arguments if el in ZZ]
                
                if(len(arguments) > 0):
                    order = max(arguments)-min(arguments)
                    print(order, end = ", ")
                    if(order >= min_order):
                        results.append(tuple([sequence, formula]))
                else:
                    print("no args", end=", ")
        print(f"\nFound {len(results)}/{min_results} so far...")
        i += 1
    return results

In [190]:
results = search_dfinite_order(5)

Cathcing 100 first sequences
Analyzing...
Found orders 1, 1, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 4, 2, 2, 1, 1, 2, 3, 4, 2, 1, 2, 2, 1, 2, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 2, 1, 2, 1, 1, 2, 1, 2, 2, 1, 2, 1, 2, 3, 2, 1, 3, 2, 2, 2, 2, 3, 4, 2, 4, 1, 2, 2, 2, 2, 2, 1, 3, 1, 7, 2, 2, 1, 3, 1, 1, 3, 1, 2, 1, 2, 1, 1, 2, 2, 2, 1, 3, 4, 1, 4, 1, 1, 1, 1, 1, 
Found 1/5 so far...
Cathcing 200 first sequences
Analyzing...
Found orders 6, 2, 1, 1, 4, 2, 3, 1, 1, 1, 1, 4, 2, 2, 1, 1, 3, 2, 1, 2, 1, 3, 3, 3, 2, 2, 1, 2, 2, 1, 2, 4, 2, 1, 1, 2, 3, 1, 1, 2, 2, 2, 2, 1, 2, 2, 2, 6, 3, 4, 1, 1, 3, 1, 1, 2, 4, 1, 3, 1, 2, 1, 3, 1, 1, 2, 2, 1, 1, 2, 3, 3, 1, 1, 1, 2, 3, 3, 2, no args, 1, 2, 2, 1, 3, 5, 2, 6, 3, 2, 1, 3, 1, 1, 1, 2, 1, 2, 1, 1, 2, 3, 2, 1, 
Found 5/5 so far...


In [123]:
[el for el in res.formulas() if el.con

/home/anton/sage/local/lib/python3.9/site-packages/docutils/io.py:245: DeprecationWarning: 'U' mode is deprecated
  self.source = open(source_path, mode, **kwargs)


In [179]:
a = "hola"

In [141]:
a.find?

In [149]:
import re

In [158]:
re.findall(r"a\(([^\)]*)\)", a)

['n', 'n-1']

In [155]:
a

'a(n) = n*a(n-1), n >= 1.'

In [180]:
a[2:0]

''